In [3]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.utils import to_categorical
import numpy as np
import string

## 1. Configuration

In [1]:
IMG_HEIGHT = 64
IMG_WIDTH = 256
BATCH_SIZE = 32
EPOCHS = 50

CHARS = "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ -"
CHAR_TO_NUM = {char: i for i, char in enumerate(CHARS)}
NUM_TO_CHAR = {i: char for i, char in enumerate(CHARS)}

## 2. Data Loader

In [6]:
# Load and preprocess image

def preprocess_image(image_path, img_height=IMG_HEIGHT, img_width=IMG_WIDTH):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=1)
    img = tf.image.resize(img, [img_height, img_width])
    img = tf.cast(img, tf.float32) / 255.0
    return img

# Encode text to integer sequence

def encode_text(text):
    return [CHAR_TO_NUM[c] for c in text if c in CHAR_TO_NUM]